In [57]:
from __future__ import division
import pandas as pd
import numpy as np
import operator
import pickle

In [58]:
from hpo_similarity import open_ontology
graph, alt_ids, obsolete_ids = open_ontology()

In [59]:
%%time
data = pd.read_excel('phenotype_original.xlsx')
g = data.groupby('#DatabaseID')
G= g['HPO_ID'].apply(lambda s: s.tolist())
OMIM_DICT = G.to_dict()

CPU times: user 6.06 s, sys: 25.6 ms, total: 6.09 s
Wall time: 6.09 s


In [60]:
%%time
graph.tally_hpo_terms(OMIM_DICT)

CPU times: user 274 ms, sys: 3.02 ms, total: 277 ms
Wall time: 277 ms


In [61]:
data1= data.head(108580)
data2= data.loc[108580:]

In [62]:
set_id = set(data["HPO_ID"])
print(len(set_id))

9357


In [63]:
def lin_score(graph, term1, term2):
    a = graph.calculate_information_content(term1)
    b = graph.calculate_information_content(term2)
    c = 2*graph.get_most_informative_ic(term1,term2)
    #print ("##@@",a,b,c)
    try:
        lin_score = c/(b + a)
    except ZeroDivisionError:
        lin_score = 0
    return lin_score

In [64]:
df_OMIM_ORPHA = pd.DataFrame(columns=['Phenotype_ID', 'Number_of_Disease', 'Information_Content','Lin_Score'])

In [65]:
children = graph.get_immediate_descendants('HP:0001371')
children

{'HP:0002803',
 'HP:0002828',
 'HP:0003121',
 'HP:0004631',
 'HP:0005781',
 'HP:0005876',
 'HP:0005997',
 'HP:0030044',
 'HP:0100492'}

In [70]:
i=0
for child in children:
    df_OMIM_ORPHA.loc[i,['Phenotype_ID']]= child
    #print (child)
    #print (graph.calculate_information_content(child))
    #print (graph.get_term_count(child))
    #print (graph.get_ids_per_term(child))
    #print (lin_score(child, 'HP:0001371'))
    df_OMIM_ORPHA.loc[i,['Number_of_Disease']]= graph.get_term_count(child)
    df_OMIM_ORPHA.loc[i,['Information_Content']]= graph.calculate_information_content(child)
    df_OMIM_ORPHA.loc[i,['Lin_Score']]= lin_score(graph, child, 'HP:0001371')
    i= i+1

In [71]:
df_OMIM_ORPHA

,Phenotype_ID,Number_of_Disease,Information_Content,Lin_Score
0,HP:0030044,417,3.339118,0.849684
1,HP:0003121,503,3.151614,0.878043
2,HP:0002803,174,4.213149,0.738503
3,HP:0100492,31,5.938217,0.586924
4,HP:0004631,7,7.426294,0.498638
5,HP:0005997,3,8.273592,0.4593
6,HP:0005876,9,7.17498,0.511636
7,HP:0005781,13,6.807255,0.531923
8,HP:0002828,42,5.634534,0.608926


In [72]:
graph1, alt_ids, obsolete_ids = open_ontology()
g1 = data1.groupby('#DatabaseID')
G1= g1['HPO_ID'].apply(lambda s: s.tolist())
OMIM_DICT1 = G1.to_dict()
graph1.tally_hpo_terms(OMIM_DICT1)

In [73]:
df_OMIM = pd.DataFrame(columns=['Phenotype_ID', 'Number_of_Disease', 'Information_Content','Lin_Score'])
i=0
for child in children:
    df_OMIM.loc[i,['Phenotype_ID']]= child
    #print (child)
    #print (graph1.calculate_information_content(child))
    #print (graph1.get_term_count(child))
    #print (graph.get_ids_per_term(child))
    #print (lin_score(graph1, child, 'HP:0001371'))
    #print("\n")
    df_OMIM.loc[i,['Number_of_Disease']]= graph1.get_term_count(child)
    df_OMIM.loc[i,['Information_Content']]= graph1.calculate_information_content(child)
    df_OMIM.loc[i,['Lin_Score']]= lin_score(graph1, child, 'HP:0001371')
    i= i+1

In [74]:
df_OMIM

,Phenotype_ID,Number_of_Disease,Information_Content,Lin_Score
0,HP:0030044,192,3.704512,0.782352
1,HP:0003121,235,3.502422,0.808258
2,HP:0002803,114,4.225809,0.722609
3,HP:0100492,16,6.189418,0.561187
4,HP:0004631,2,8.26886,0.453827
5,HP:0005997,3,7.863395,0.471412
6,HP:0005876,3,7.863395,0.471412
7,HP:0005781,3,7.863395,0.471412
8,HP:0002828,10,6.659422,0.532703
